# Running the Micom Workflow for the Binary Azotobacter - Rhodosporidium Model

In this notebook we utilize the package `micom` to generate a binary community model for 2 organisms of interest:
- `Azotobacter vinelandii`
- `Rhodosporidium toruloides`

This binary consortium allows us to gain insights into the exchanges between the 2 organisms and run FBA experiments.

First off we can import all necessary packages for this notebook.

In [171]:
import pandas as pd
import cobra

from micom import Community
from micom.workflows import build, grow, tradeoff, fix_medium,build_database
from micom import load_pickle
from micom.viz import plot_tradeoff, plot_exchanges_per_sample, plot_growth

import os
os.environ["GRB_LICENSE_FILE"]

'/Users/mcna892/Desktop/Projects/Digital_Twins/gurobi.lic'

## Setting Up the Model in MICOM

To begin, we need to import our genome-scale models into `micom`. We have these models saved as `.sbml` files as this form is accepted easily into programs such as `cobrapy` and `micom`.

### Building a Taxonomy

Step #1: Establish a Taxonomy that lists the out the taxonomy for our organisms of interest

In [486]:
Tax= pd.DataFrame(columns=['id','genus','species','reactions','metabolites','sample_id','abundance'])
Tax.loc[len(Tax.index)] = ['Azotobacter','Azotobacter','A. vinelandii',2469,2003,'One',100]
Tax.loc[len(Tax.index)] = ['Rhodosporidium', 'Rhodosporidium', 'R. toruloides',2398,2051,'One',900]
Tax

,id,genus,species,reactions,metabolites,sample_id,abundance
0,Azotobacter,Azotobacter,A. vinelandii,2469,2003,One,100
1,Rhodosporidium,Rhodosporidium,R. toruloides,2398,2051,One,900


This taxonomy file lists important information for `micom` down the road, such as the number of `reactions` and `metabolites` in the provided models.

### Building a Database

Step #2: Now we must construct a database for the `.sbml` models to be preprocessed and stored. This is done by supplying `micom` with a file which contains model path locations.

In [487]:
db = pd.read_csv('./man_av_rt.csv')
db

,file,kingdom,phylum,class,order,family,genus,species
0,./azo_vine.xml,bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Azotobacter,A. vinelandii
1,./Rt_IFO0880.xml,fungi,a,Ustilaginomycetes,b,c,Rhodosporidium,R. toruloides


In [488]:
build_database(db,'./db_av_rt')

/Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

,file,kingdom,phylum,class,order,family,genus,species,id,summary_rank
genus,,,,,,,,,,
Azotobacter,Azotobacter.json,bacteria,Pseudomonadota,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Azotobacter,A. vinelandii,Azotobacter,genus
Rhodosporidium,Rhodosporidium.json,fungi,a,Ustilaginomycetes,b,c,Rhodosporidium,R. toruloides,Rhodosporidium,genus


In [489]:
db_path = './db_av_rt' 

### Construct Manifest Object

Step #3: Now that we have the Taxonomy and Database constructed we can build our community model. This is done by using the `build()` method in `micom`.

__Note__:

Build Manifest object from Taxonomy DataFrame and the corresponding database directory

Skip this step if manifest has already been built and saved to "models" directory


__IMPORTANT__: Declare the Solver you would like to use for this Community model here:
- osqp (good for smaller models)
- gurobi
- glpk
- cplex
- scipy


In [490]:
manifest = build(Tax, out_folder="models_av_rt", model_db=db_path, cutoff=0.0001, threads=10,solver='gurobi')
manifest

[17:34:45] WARNING  Found existing models for 1 samples. Will skip those. Delete the output folder if   ]8;id=971905;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/workflows/build.py\build.py]8;;\:]8;id=524118;file:///Users/mcna892/opt/anaconda3/envs/micom/lib/python3.9/site-packages/micom/workflows/build.py#95\95]8;;\
                    you would like me to rebuild them.                                                             

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmp84efd8gj.lp
Reading time = 0.05 seconds
: 4506 rows, 10633 columns, 41351 nonzeros


,sample_id,file,found_taxa,total_taxa,found_fraction,found_abundance_fraction
0,One,One.pickle,2.0,2.0,1.0,1.0


## Running the Models with FBA

Now that we have the `manifest`, we can load the model as a `Community` object through `micom`. This will give us some functionality similar to that of `cobrapy`. This can be done with the `load_pickle()` method we imported above through `micom`.

In [177]:
community = load_pickle("models_av_rt/One.pickle")
print(len(community.reactions))

Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmpgjn1mw7r.lp
Reading time = 0.03 seconds
: 4506 rows, 10633 columns, 41351 nonzeros
5316


### Exploring the Model Attributes

Our new variable `community` behaves very similarly to a standard `cobrapy` model. We can explore it's attributes in a similar way as well.

Things such as `reactions` and `metabolites`:

In [178]:
community.reactions

[<Reaction ALATA_D2__Azotobacter at 0x3b151f1c0>,
 <Reaction SHCHD2__Azotobacter at 0x3b151f370>,
 <Reaction CPPPGO__Azotobacter at 0x3b151f700>,
 <Reaction GTHOr__Azotobacter at 0x3b135bc70>,
 <Reaction DHORD5__Azotobacter at 0x3b0dd4b80>,
 <Reaction GLYCTO2__Azotobacter at 0x3b151fb80>,
 <Reaction GLYCTO3__Azotobacter at 0x3b18ddf10>,
 <Reaction GLYCTO4__Azotobacter at 0x3b0dfd6a0>,
 <Reaction TRPS2__Azotobacter at 0x3b174aeb0>,
 <Reaction G3PD5__Azotobacter at 0x3b1760e20>,
 <Reaction EX_cellb_e__Azotobacter at 0x3b0dfd3d0>,
 <Reaction EX_chol_e__Azotobacter at 0x3b1776f40>,
 <Reaction LEUTAi__Azotobacter at 0x3b0d7c280>,
 <Reaction SHK3Dr__Azotobacter at 0x3b1776a00>,
 <Reaction G5SD__Azotobacter at 0x3ab638b80>,
 <Reaction ALATA_L2__Azotobacter at 0x3ab638a30>,
 <Reaction APRAUR__Azotobacter at 0x3ab638640>,
 <Reaction DB4PS__Azotobacter at 0x3ab6389a0>,
 <Reaction RBFK__Azotobacter at 0x3ab758490>,
 <Reaction ACP1_FMN__Azotobacter at 0x3ab67e070>,
 <Reaction RBFSb__Azotobacter at

In [179]:
community.metabolites

[<Metabolite ala__D_c__Azotobacter at 0x3b151f1f0>,
 <Metabolite pyam5p_c__Azotobacter at 0x3b151f220>,
 <Metabolite pydx5p_c__Azotobacter at 0x3b151f250>,
 <Metabolite pyr_c__Azotobacter at 0x3b151f280>,
 <Metabolite dscl_c__Azotobacter at 0x3b151f3a0>,
 <Metabolite h_c__Azotobacter at 0x3b151f310>,
 <Metabolite nad_c__Azotobacter at 0x3b151f2b0>,
 <Metabolite nadh_c__Azotobacter at 0x3b151f2e0>,
 <Metabolite scl_c__Azotobacter at 0x3b151f430>,
 <Metabolite co2_c__Azotobacter at 0x3b151f790>,
 <Metabolite cpppg3_c__Azotobacter at 0x3b151f640>,
 <Metabolite h2o_c__Azotobacter at 0x3b151f5b0>,
 <Metabolite o2_c__Azotobacter at 0x3b151f550>,
 <Metabolite pppg9_c__Azotobacter at 0x3b151f400>,
 <Metabolite gthox_c__Azotobacter at 0x3b151f910>,
 <Metabolite gthrd_c__Azotobacter at 0x3b151f970>,
 <Metabolite nadp_c__Azotobacter at 0x3b151f6d0>,
 <Metabolite nadph_c__Azotobacter at 0x3b151f460>,
 <Metabolite dhor__S_c__Azotobacter at 0x3b18d1250>,
 <Metabolite mql8_c__Azotobacter at 0x3b0dded

and importantly the `medium`

In [180]:
community.medium

{'EX_pi_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_glc__D_m': 5.0,
 'EX_zn2_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_cobalt2_m': 999999.0,
 'EX_sel_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_nh4_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_tungs_m': 999999.0,
 'EX_slnt_m': 999999.0}

This behavior mimics the medium in `cobrapy`, but combines both models mediums into 1

### Running Optimization

Now that we have the model loaded, we can run standard `FBA` methods using `optimize()`. Default optimize does not return any fluxes from the model, so we can set the `fluxes=True` when calling the method to return them.

In [181]:
result = community.optimize(fluxes=True)
result

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.5,0.880685,2469,2003
Rhodosporidium,0.5,0.000000,2398,2051
medium,NaN,NaN,449,449


We can see that both organisms have a non-zero growth rate and that the community growth is also non-zero. Let's check the fluxes.

In [425]:
result.fluxes.T.loc['EX_glyc__R_e']

compartment
Azotobacter      -57.384459
Rhodosporidium    57.384459
medium                  NaN
Name: EX_glyc__R_e, dtype: float64

#### Changing parts of the medium to test it's effect on growth

Now that we can successfully optimize the community model, we can begin altering the models medium and seeing how it changes the (community) growth rate.

First, let's make a copy of the original medium so that we can restore it after making changes.

In [260]:
medium_bkp = community.medium

Now we can make changes to the medium. The following cell is meant to be re-run with making changes. It will first restore the medium to the original and them set 

In [621]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
medium_to_change = community.medium

#Add or subtract reactions
medium_to_change["EX_xyl__D_m"] = 5
medium_to_change["EX_glc__D_m"] = 0
#medium_to_change["EX_glyc__R_m"] = 0
medium_to_change["EX_nh4_m"] = 0
medium_to_change["EX_n2_m"] = 5


# Set the new medium as the model's medium
community.medium = medium_to_change
community.medium

{'EX_pi_m': 999999.0,
 'EX_h_m': 999999.0,
 'EX_fe3_m': 999999.0,
 'EX_mn2_m': 999999.0,
 'EX_fe2_m': 999999.0,
 'EX_xyl__D_m': 5,
 'EX_zn2_m': 999999.0,
 'EX_mg2_m': 999999.0,
 'EX_ca2_m': 999999.0,
 'EX_ni2_m': 999999.0,
 'EX_cu2_m': 999999.0,
 'EX_cobalt2_m': 999999.0,
 'EX_sel_m': 999999.0,
 'EX_h2o_m': 999999.0,
 'EX_mobd_m': 999999.0,
 'EX_so4_m': 999999.0,
 'EX_k_m': 999999.0,
 'EX_na1_m': 999999.0,
 'EX_o2_m': 999999.0,
 'EX_cl_m': 999999.0,
 'EX_tungs_m': 999999.0,
 'EX_slnt_m': 999999.0,
 'EX_n2_m': 5}

Now that the medium is changed, we can rerun the model optimization.

In [622]:
result_altered_medium = community.optimize(fluxes=True, pfba=True)

In [623]:
result_altered_medium

,abundance,growth_rate,reactions,metabolites
compartments,,,,
Azotobacter,0.5,0.000000,2469,2003
Rhodosporidium,0.5,0.540935,2398,2051
medium,NaN,NaN,449,449


In [624]:
result_altered_medium.fluxes.T.loc['EX_glyc__R_e']

compartment
Azotobacter      -12.927707
Rhodosporidium    12.927707
medium                  NaN
Name: EX_glyc__R_e, dtype: float64

In [625]:
result_altered_medium.fluxes.T.loc['EX_glc__D_e']

compartment
Azotobacter       0.0
Rhodosporidium    0.0
medium            NaN
Name: EX_glc__D_e, dtype: float64

In [635]:
result_altered_medium.fluxes.T.loc['EX_co2_m']

compartment
Azotobacter             NaN
Rhodosporidium          NaN
medium            12.111542
Name: EX_co2_m, dtype: float64

In [627]:
x = result_altered_medium.fluxes.T.loc[result_altered_medium.fluxes.T.Azotobacter.index.str.startswith('EX_')]

In [638]:
x.fillna(0,inplace=True)
HTML(x[(x.medium != 0) | (x.medium != 0)].sort_values('medium').to_html())

compartment,Azotobacter,Rhodosporidium,medium
reaction,,,
EX_o2_m,0.0,0.0,-11.171455
EX_xyl__D_m,0.0,0.0,-5.000000
EX_n2_m,0.0,0.0,-0.796802
EX_k_m,0.0,0.0,-0.158414
EX_pi_m,0.0,0.0,-0.072003
EX_so4_m,0.0,0.0,-0.027111
EX_mg2_m,0.0,0.0,-0.016692
EX_na1_m,0.0,0.0,-0.007059
EX_fe3_m,0.0,0.0,-0.001669


In [639]:
x.fillna(0,inplace=True)
HTML(x[(x.Azotobacter != 0) | (x.Rhodosporidium != 0)].sort_values('Azotobacter').to_html())

compartment,Azotobacter,Rhodosporidium,medium
reaction,,,
EX_glyc__R_e,-12.927707,12.927707,0.0
EX_h_e,-7.405386,7.405386,0.0
EX_glcn_e,-6.738459,6.738459,0.0
EX_o2_e,-5.807500,-16.535409,0.0
EX_n2_e,-1.593605,0.000000,0.0
EX_pser__L_e,-0.317509,0.317509,0.0
EX_asp__L_e,-0.151047,0.151047,0.0
EX_glyclt_e,-0.002111,0.002111,0.0
EX_xylt_e,-0.002008,0.002008,0.0


In [600]:
from IPython.display import HTML

### Testing Abundances

The following code is an inline way to change the abundances

In [560]:
community.set_abundance([1,1],normalize=True)

## Running the Models with MICOM Grow

An alternative to running standard community optimization with `optimize()`, we can also use a `micom.workflows` method called `grow()`. This simulates growth of the organism while also simulating potential tradeoffs (between prioritizing community vs. individual growth). This method does not require our previously constructed `community` object, but rather the `manifest` we added earlier.

A key difference here though, is that we need to create a `DataFrame` detailing the reaction, flux, and metabolite as the medium provided to the method.

### Building the Medium

In [38]:
# Restore medium to original
community.medium = medium_bkp

# Set variable to become new medium
grow_medium_to_change = community.medium

#Add or subtract reactions
#grow_medium_to_change["EX_glc__D_m"] = 0
#grow_medium_to_change["EX_sucr_m"] = 1
grow_medium_to_change["EX_nh4_m"] = 0


In [39]:
grow_medium = pd.Series(grow_medium_to_change).to_frame('flux').reset_index()
grow_medium = grow_medium.rename(columns={'index':'reaction'})
grow_medium

,reaction,flux
0,EX_hco3_m,1.99
1,EX_mn2_m,999999.00
2,EX_mg2_m,999999.00
3,EX_ca2_m,999999.00
4,EX_nh4_m,0.00
5,EX_fe2_m,0.05
6,EX_cu2_m,999999.00
7,EX_k_m,999999.00
8,EX_h2o_m,999999.00
9,EX_o2_m,999999.00


In [40]:
result_grow = grow(manifest, model_folder="models_av_se", medium=grow_medium, tradeoff=0.01, threads=2,presolve=True)

Set parameter TokenServer to value "leghorn.emsl.pnl.gov"
Read LP format model from file /var/folders/1f/ksln774x1hd1pzfgsjgpxt7r0000gn/T/tmp8nq38lme.lp
Reading time = 0.03 seconds
: 3116 rows, 7325 columns, 28585 nonzeros


In [191]:
result_grow.exchanges

NameError: name 'result_grow' is not defined

In [42]:
result_grow.exchanges.to_csv('av_se_out.csv')

## Useful Utility Functions

In [ ]:
def medium2extracellular(medium: dict):
    
    return dict([(k[:-1] + "e",v) for k,v in medium.items()])

In [ ]:
def extracellular2medium(medium: dict):
    
    return dict([(k[:-1] + "m",v) for k,v in medium.items()])  

## CobraPy Models for Checking

In [185]:
model_azo = cobra.io.read_sbml_model('azo_vine.xml')
model_rhodo = cobra.io.read_sbml_model('Rt_IFO0880.xml')

In [555]:
med = model_azo.medium

med['EX_xyl__D_e'] = 0
med['EX_glc__D_e'] = 5

model_azo.medium = med

model_azo.optimize().objective_value

0.40844618988920894

In [642]:
model_azo.reactions.BIOMASS_Av_DJ_core.build_reaction_string()

'0.000223 10fthf_c + 2.6e-05 2fe2s_c + 0.000223 2ohph_c + 0.00026 4fe4s_c + 0.583020414344392 ala__L_c + 0.000223 amet_c + 0.4009934072884 arg__L_c + 0.126569944053473 asn__L_c + 0.267095137584612 asp__L_c + 75.55223 atp_c + 2e-06 btn_c + 0.005205 ca2_c + 0.005205 cl_c + 0.000576 coa_c + 2.5e-05 cobalt2_c + 0.133508 ctp_c + 0.000709 cu2_c + 0.0524198607691142 cys__L_c + 0.026166 datp_c + 0.027017 dctp_c + 0.027017 dgtp_c + 0.026166 dttp_c + 0.000223 fad_c + 0.006715 fe2_c + 0.007808 fe3_c + 0.194867292315076 gln__L_c + 0.31452909011218 glu__L_c + 0.429800509517426 gly_c + 0.215096 gtp_c + 70.028756 h2o_c + 0.116750318531968 his__L_c + 0.219621483067065 ile__L_c + 0.195193 k_c + 0.019456 kdo2lipid4_e + 0.613524776645025 leu__L_c + 0.140105014137193 lys__L_c + 0.102844696541777 met__L_c + 0.008675 mg2_c + 0.000223 mlthf_c + 0.000691 mn2_c + 7e-06 mobd_c + 0.013894 murein5px4p_p + 0.001831 nad_c + 0.000447 nadp_c + 0.013013 nh4_c + 0.000323 ni2_c + 0.063814 pe160_p + 0.075214 pe161_p + 0.